# Udemy講座「LangChainによる大規模言語モデル（LLM）アプリケーション開発入門」セクション「（アップデート）LangChain Expression Language （LCEL）」のソースコード

## LangChain Expression Language （LCEL）の概要と実装例

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install --quiet langchain==0.0.329 openai==0.28.1

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# PromptTemplateを準備
prompt = PromptTemplate.from_template("""料理のレシピを考えてください。

料理名: {dish}""")

# LLMを準備
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chainを作成
chain = prompt | llm

# Chainを作成する処理は、LCEL以前は以下のように書いていました。
# chain = LLMChain(llm=chat, prompt=prompt)

result = chain.invoke({"dish": "カレー"})
print(result.content)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

# OutputParserを準備
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

# PromptTemplateを準備
prompt = PromptTemplate.from_template("""料理のレシピを考えてください。

{format_instructions}

料理名: {dish}""", partial_variables = {"format_instructions": output_parser.get_format_instructions()})

# LLMを準備
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chainを作成
chain = prompt | llm | output_parser

# Chainを実行
result = chain.invoke({"dish": "カレー"})
print(type(result))
print(result)